## BERT Based Sentiment Analysis Model Server
The model used here, was trained with the concept of transfer learning  i.e. taking huggingface transformers pretrained BERT model and further training it on a custom dataset of reviews. this yields a sentiment analysis model based on the prior knowledge of BERT. 
The model server is given a list of texts and outputs a list of labels corresponding to its prediction.
The labels express the sentiment of the writer towards the topic of the text:
0 for negative sentiment, 1 for neutral and 2 for positive.

The model file (~430 MB), can be downloaded to your local environment from: https://iguazio-sample-data.s3.amazonaws.com/models/model.pt

## mlconfig

In [1]:
%load_ext lab_black
from mlrun import mlconf
from bert_sentiment_analysis_serving import (
    BertSentimentClassifier,
    SentimentClassifierServing,
)
import os

mlconf.dbpath = mlconf.dbpath or "http://mlrun-api:8080"
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

## Examples locally
You may change model_dir to point at the path where model.pt file is saved

In [2]:
model_dir = "/User/demo_stocks/artifacts/models/bert_sentiment_analysis_model.pt"
model_server = SentimentClassifierServing("model-server", model_dir=model_dir)
model_server.load()

### example 1
Here we test a pretty straightforward example for positive sentiment.

In [3]:
output = model_server.predict(
    {
        "instances": [
            "I had a pleasure to work with such dedicated team. Looking forward to \
             cooperate with each and every one of them again."
        ]
    }
)

assert output[0] == 2

### example 2
Now we will test a couple more examples. These are arguably harder due to misleading words that express, on their own, an opposite sentiment comparing to the full text. 

In [4]:
output = model_server.predict(
    {
        "instances": [
            "This app is amazingly useless.",
            "As much as I hate to admit it, the new added feature is surprisingly user friendly.",
        ]
    }
)

assert output[0] == 0
assert output[1] == 2

## Remote activation
Create a function object with custom specification.

In [5]:
from mlrun import new_model_server, mount_v3io
import requests
import yaml

with open("item.yaml") as item_file:
    items = yaml.load(item_file, Loader=yaml.FullLoader)

In [6]:
fn = new_model_server(
    name=items["name"],
    model_class="SentimentClassifierServing",
    filename=items["spec"]["filename"],
)

fn.spec.description = items["description"]
fn.metadata.categories = items["categories"]
fn.metadata.labels = labels = items["labels"]
fn.spec.image = items["spec"]["image"]
fn.spec.requirements = ["transformers==3.0.2"]
fn.spec.handler = items["spec"]["handler"]
fn.spec.max_replicas = 1
fn.spec.readiness_timeout = 500
fn.export("bert_sentiment_analysis_serving.yaml")

fn.add_model(
    "bert_classifier_v1",
    "/User/demo_stocks/artifacts/models/bert_sentiment_analysis_model.pt",
)

> 2021-02-17 18:17:51,836 [info] function spec saved to path: bert_sentiment_analysis_serving.yaml


In [7]:
if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io

    fn.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at
    # https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc

    fn.apply(mount_pvc("nfsvol", "nfsvol", "/home/joyan/data"))

In [8]:
addr = fn.deploy(project='nlp-servers')

> 2021-02-17 18:17:51,929 [info] Starting remote function deploy
2021-02-17 18:17:52  (info) Deploying function
2021-02-17 18:17:52  (info) Building
2021-02-17 18:17:52  (info) Staging files and preparing base images
2021-02-17 18:17:52  (info) Building processor image
2021-02-17 18:17:53  (info) Build complete
Failed to deploy. Details:
/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729006826/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Trigger

RunError: cannot deploy Failed to deploy. Details:
/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729006826/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729006826/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please

## Remote example
We will send a sentence to the model server via HTTP request. Note that the url below uses model server notation that directs our event to the predict function.

In [ ]:
import json

event_data = {'instances': ['I had a somewhat ok experience buying at that store.']}

resp = requests.put(addr + '/bert_classifier_v1/predict', json=json.dumps(event_data))

In [ ]:
print(resp.text)

The model server classified the sentence as neutral. 